# Collision Avoidance - Live Demo

In this notebook we'll use the model we trained to detect whether the robot is ``free`` or ``blocked`` to enable a collision avoidance behavior on the robot.  

## Load the trained model

We'll assumed that you've already downloaded the ``best_model.pth`` to your workstation as instructed in the training notebook.  Now, you should upload this model into this notebook's
directory by using the Jupyter Lab upload tool.  Once that's finished there should be a file named ``best_model.pth`` in this notebook's directory.  

> Please make sure the file has uploaded fully before calling the next cell

Execute the code below to initialize the PyTorch model.  This should look very familiar from the training notebook.

In [1]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

Next, load the trained weights from the ``best_model.pth`` file that you uploaded

In [2]:
model.load_state_dict(torch.load('best_model_c_stop.pth'))

Currently, the model weights are located on the CPU memory execute the code below to transfer to the GPU device.

In [3]:
device = torch.device('cuda')
model = model.to(device)

### Create the preprocessing function

We have now loaded our model, but there's a slight issue.  The format that we trained our model doesnt *exactly* match the format of the camera.  To do that, 
we need to do some *preprocessing*.  This involves the following steps

1. Convert from BGR to RGB
2. Convert from HWC layout to CHW layout
3. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
4. Transfer the data from CPU memory to GPU memory
5. Add a batch dimension

In [4]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

Great! We've now defined our pre-processing function which can convert images from the camera format to the neural network input format.

Now, let's start and display our camera.  You should be pretty familiar with this by now.  We'll also create a slider that will display the
probability that the robot is blocked.

In [5]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, blocked_slider]))

We'll also create our robot instance which we'll need to drive the motors.

In [6]:
from jetbot import Robot

robot = Robot()

Next, we'll create a function that will get called whenever the camera's value changes.  This function will do the following steps

1. Pre-process the camera image
2. Execute the neural network
3. While the neural network output indicates we're blocked, we'll turn left, otherwise we go forward.

In [7]:
import torch.nn.functional as F
import time

def update(change):
    global blocked_slider, robot
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    print(y)
    
    prob_blocked = float(y.flatten()[0])
    
    blocked_slider.value = prob_blocked
    
    if prob_blocked < 0.5:
        robot.forward(0.4)
    else:
        robot.stop()
    
    time.sleep(0.001)
        
update({'new': camera.value})  # we call the function once to intialize

tensor([[8.8636e-07, 1.0000e+00]], device='cuda:0', grad_fn=<SoftmaxBackward>)


Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing. 

We accomplish that with the ``observe`` function.

> WARNING: This code will move the robot!! Please make sure your robot has clearance.  The collision avoidance should work, but the neural
> network is only as good as the data it's trained on!

In [8]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

tensor([[4.0885e-05, 9.9996e-01]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([[0.0031, 0.9969]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([[2.7949e-04, 9.9972e-01]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([[7.2659e-05, 9.9993e-01]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([[3.3216e-05, 9.9997e-01]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([[9.4463e-04, 9.9906e-01]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([[9.0016e-04, 9.9910e-01]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([[0.0012, 0.9988]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([[0.0044, 0.9956]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([[8.8569e-04, 9.9911e-01]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([[0.0470, 0.9530]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([[0.0061, 0.9939]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([[0.0043, 0.9957]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([[0.1489, 0.85

Awesome! If your robot is plugged in it should now be generating new commands with each new camera frame.  Perhaps start by placing your robot on the ground and seeing what it does when it reaches an obstacle.

If you want to stop this behavior, you can unattach this callback by executing the code below.

In [18]:
camera.unobserve(update, names='value')
robot.stop()

Perhaps you want the robot to run without streaming video to the browser.  You can unlink the camera as below.

In [ ]:
camera_link.unlink()  # don't stream to browser (will still run camera)

To continue streaming call the following.

In [ ]:
camera_link.link()  # stream to browser (wont run camera)

### Conclusion

That's it for this live demo!  Hopefully you had some fun and your robot avoided collisions intelligently! 

If your robot wasn't avoiding collisions very well, try to spot where it fails.  The beauty is that we can collect more data for these failure scenarios
and the robot should get even better :)